In [49]:
 # %matplotlib inline
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
##from wordcloud import WordCloud

In [56]:
hm_data = pd.read_csv('cleaned_hm.csv')
hm_data.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


In [45]:
hm_data.count()

hmid                     100535
wid                      100535
reflection_period        100535
original_hm              100535
cleaned_hm               100535
modified                 100535
num_sentence             100535
ground_truth_category     14125
predicted_category       100535
dtype: int64

In [57]:
hm_data.predicted_category.unique()

array(['affection', 'exercise', 'bonding', 'leisure', 'achievement',
       'enjoy_the_moment', 'nature'], dtype=object)

In [58]:
hm_data.loc[hm_data["reflection_period"]=='3m'].head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
30455,58297,888,3m,nowing im going to travel to china soon,nowing im going to travel to china soon,True,1,leisure,leisure
30456,58298,129,3m,I ate a really good dinner.,I ate a really good dinner.,True,1,NaN,enjoy_the_moment
30457,58299,1354,3m,I went to a Lehigh Valley phantoms hockey game.,I went to a Lehigh Valley phantoms hockey game.,True,1,NaN,leisure
30458,58300,384,3m,I saw the woman I love.,I saw the woman I love.,True,1,NaN,affection
30459,58301,1111,3m,the babyset i ordered finally came and i set i...,the baby set i ordered finally came and i set ...,False,1,NaN,achievement


In [59]:
df_hm = hm_data[hm_data['cleaned_hm'].notnull()]
len_count = df_hm['cleaned_hm'].apply(lambda x: len(x.split()))
len_count.describe()


TypeError: 'int' object is not callable

In [55]:
df_hm[df_hm['cleaned_hm'].apply(lambda x: len(x.split()))==2].head()

TypeError: 'int' object is not callable

In [60]:
length_order = ["0-4", "5-9", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", \
                "40-44", "45-49", ">=50"]
length_category = len_count.apply(lambda x: length_order[min(10, int(x/5))])
length_counts = pd.DataFrame(length_category.value_counts()).reset_index()
length_counts.columns = ['word numbers', '# of moments']

sns.barplot(x='word numbers', y='# of moments', data=length_counts, order=length_order)

In [ ]:
#text = ' '.join(df_hm['cleaned_hm'].tolist())
#text = text.lower()
#wordcloud = WordCloud(background_color="white", height=2700, width=3600).generate(text)
#plt.figure( figsize=(14,8) )
#plt.imshow(wordcloud.recolor(colormap=plt.get_cmap('Set2')), interpolation='bilinear')
#plt.axis("off")

In [61]:
seasons = ['Spring', 'Summer', 'Fall', 'Winter']

# Check each moment, and increase the count for the mentioned season
season_dic = dict((x,0) for x in seasons)
tokens_hm = df_hm['cleaned_hm'].apply(lambda x: x.split())
for _, value in tokens_hm.iteritems():
    for word in value:
        if word in seasons:
            season_dic[word] += 1
            
season_dic

{'Fall': 6, 'Spring': 81, 'Summer': 27, 'Winter': 6}

In [62]:
# Read the happyDB sample file
with open('happydb.txt', 'w') as ofile:
    len = int(df_hm.shape[0] / 8)
    for i in range(0, len - 1):
        ofile.write(df_hm['cleaned_hm'].iloc[i] + '\n')
        
print("Happy moments are retrieved!")

Happy moments are retrieved!


In [63]:
df1 = pd.read_csv('demographic.csv')

In [64]:
df1.count()

wid           10844
age           10809
country       10771
gender        10812
marital       10787
parenthood    10813
dtype: int64

In [65]:
# to combine two datasets we can use merge or append
hm_new = pd.merge(hm_data , df1)

In [66]:
hm_new.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category,age,country,gender,marital,parenthood
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection,35,USA,m,single,n
1,27873,2053,24h,I played a new game that was fun and got to en...,I played a new game that was fun and got to en...,True,1,NaN,leisure,35,USA,m,single,n
2,28073,2053,24h,I listened to some music and heard an entire a...,I listened to some music and heard an entire a...,True,1,NaN,leisure,35,USA,m,single,n
3,33522,2053,24h,Went to see a movie with my friend,Went to see a movie with my friend,True,1,NaN,bonding,35,USA,m,single,n
4,34522,2053,24h,"Played guitar, learning a song on it","Played guitar, learning a song on it",True,1,NaN,leisure,35,USA,m,single,n


In [67]:
hm_new.count()

hmid                     100535
wid                      100535
reflection_period        100535
original_hm              100535
cleaned_hm               100535
modified                 100535
num_sentence             100535
ground_truth_category     14125
predicted_category       100535
age                      100442
country                  100332
gender                   100456
marital                  100378
parenthood               100457
dtype: int64

In [68]:
hm_data.count()

hmid                     100535
wid                      100535
reflection_period        100535
original_hm              100535
cleaned_hm               100535
modified                 100535
num_sentence             100535
ground_truth_category     14125
predicted_category       100535
dtype: int64

In [69]:
hm_new

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category,age,country,gender,marital,parenthood
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection,35,USA,m,single,n
1,27873,2053,24h,I played a new game that was fun and got to en...,I played a new game that was fun and got to en...,True,1,NaN,leisure,35,USA,m,single,n
2,28073,2053,24h,I listened to some music and heard an entire a...,I listened to some music and heard an entire a...,True,1,NaN,leisure,35,USA,m,single,n
3,33522,2053,24h,Went to see a movie with my friend,Went to see a movie with my friend,True,1,NaN,bonding,35,USA,m,single,n
4,34522,2053,24h,"Played guitar, learning a song on it","Played guitar, learning a song on it",True,1,NaN,leisure,35,USA,m,single,n
5,35522,2053,24h,Talked to my romantic prospect,Talked to my romantic prospect,True,1,NaN,affection,35,USA,m,single,n
6,40281,2053,24h,I played a game for about half an hour.,I played a game for about half an hour.,True,1,leisure,leisure,35,USA,m,single,n
7,40381,2053,24h,I discovered another album I hadn't heard by a...,I discovered another album I hadn't heard by a...,True,1,NaN,achievement,35,USA,m,single,n
8,40481,2053,24h,I talked to the girl I'm dating about some int...,I talked to the girl I'm dating about some int...,True,1,NaN,affection,35,USA,m,single,n
9,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection,29.0,IND,m,married,y


In [70]:
# Drop features
hm_new.drop('original_hm', axis=1, inplace=True)

In [71]:
# Check if its dropped
hm_new.head()

,hmid,wid,reflection_period,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category,age,country,gender,marital,parenthood
0,27673,2053,24h,I went on a successful date with someone I fel...,True,1,NaN,affection,35,USA,m,single,n
1,27873,2053,24h,I played a new game that was fun and got to en...,True,1,NaN,leisure,35,USA,m,single,n
2,28073,2053,24h,I listened to some music and heard an entire a...,True,1,NaN,leisure,35,USA,m,single,n
3,33522,2053,24h,Went to see a movie with my friend,True,1,NaN,bonding,35,USA,m,single,n
4,34522,2053,24h,"Played guitar, learning a song on it",True,1,NaN,leisure,35,USA,m,single,n


In [72]:
# Drop multiple features at once in a list
hm_new.drop(['cleaned_hm','modified','num_sentence','ground_truth_category','reflection_period','hmid', 'wid'], axis=1, inplace=True)

In [73]:
hm_new

,predicted_category,age,country,gender,marital,parenthood
0,affection,35,USA,m,single,n
1,leisure,35,USA,m,single,n
2,leisure,35,USA,m,single,n
3,bonding,35,USA,m,single,n
4,leisure,35,USA,m,single,n
5,affection,35,USA,m,single,n
6,leisure,35,USA,m,single,n
7,achievement,35,USA,m,single,n
8,affection,35,USA,m,single,n
9,affection,29.0,IND,m,married,y


In [74]:
print(type(hm_new))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
dataset = hm_new.as_matrix()

In [77]:
hm_dataset = np.append(arr = dataset[:, 1:], values = dataset[:, [0]], axis = 1)

In [78]:
hm_dataset

array([['35', 'USA', 'm', 'single', 'n', 'affection'],
       ['35', 'USA', 'm', 'single', 'n', 'leisure'],
       ['35', 'USA', 'm', 'single', 'n', 'leisure'],
       ..., 
       ['65', 'IND', 'm', 'married', 'y', 'achievement'],
       ['65', 'IND', 'm', 'married', 'y', 'bonding'],
       ['65', 'IND', 'm', 'married', 'y', 'enjoy_the_moment']], dtype=object)

In [ ]:
X = hm_new.iloc[:, []]